In [1]:
import IPython
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
html_string = requests.get('https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyDataTable.aspx').content

In [3]:
soup = BeautifulSoup(html_string, 'html.parser')

In [4]:
table = soup.find_all("tr", class_="ms-rteTableEvenRow-3")
SB = table[21]

In [5]:
# Avg # tests per day (per 100,000 population)
daily_tests = SB.find_all("td", class_="ms-rteTableOddCol-3")[0].text

# Case rate per 100,000
case_rate = SB.find_all("td", class_="ms-rteTableEvenCol-3")[1].text

# Testing positivity (%)
test_positivity = SB.find_all("td", class_="ms-rteTableOddCol-3")[1].text

# % Change in 3-day avg COVID+ hospitalized patients
change_hospit = SB.find_all("td", class_="ms-rteTableEvenCol-3")[2].text

# % ICU beds currently available
ICU_beds = SB.find_all("td", class_="ms-rteTableOddCol-3")[2].text

# % Ventilators currently available
ventilators = SB.find_all("td", class_="ms-rteTableEvenCol-3")[3].text

In [6]:
# table = pd.DataFrame({"Date": [date.today().strftime("%y/%m/%d")],
#                       "Daily tests (per 100,000)": [daily_tests], 
#                       "Case rate (per 100,000)": [case_rate],
#                       "Testing positivity (%)": [test_positivity],
#                       "Change in hospitalizations (%)": [change_hospit], 
#                       "ICU beds available (%)": [ICU_beds],
#                       "Ventilators available (%)": [ventilators]})
# table.set_index("Date")

In [7]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

In [8]:
sheet = client.open("COVID Daily Metrics").sheet1
insertRow = [datetime.date.today().strftime("%y/%m/%d"),
             daily_tests,
             case_rate,
             test_positivity,
             change_hospit,
             ICU_beds,
             ventilators]
sheet.insert_row(insertRow, 2)

{'spreadsheetId': '13GG0FPUFg_5xCouN0ZF3woaPXwjuxL2Dw3uYIfwUW-Q',
 'updatedRange': 'Sheet1!A2:G2',
 'updatedRows': 1,
 'updatedColumns': 7,
 'updatedCells': 7}

In [9]:
sheet = client.open("COVID Daily Metrics").sheet1
data = sheet.get_all_records()
result = pd.DataFrame(data).set_index("Date")

In [10]:
result

,"Daily tests (per 100,000)","Case rate (per 100,000)",Testing positivity (%),Change in hospitalizations (%),ICU beds available (%),Ventilators (available (%)
Date,,,,,,
20/08/20,214.5,148.8,✔,✔,✔,✔
20/08/20,214.5,148.8,✔,✔,✔,✔
20/08/20,214.5,148.8,✔,✔,✔,✔


In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:
! jupyter nbconvert "index.ipynb" --to html --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['remove_cell']" --TagRemovePreprocessor.remove_input_tags="['remove_input']" --no-prompt

In [ ]:
now = datetime.datetime.now()
commit_message = "Last scraped on " + str(now)

In [ ]:
! git add .
! git commit -m commit_message
! git push